## Trigger Warning: мы использовали смайлики специально - для красоты


In [6]:
import streamlit as st
import plotly.express as px
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import chardet

In [8]:
st.set_page_config(page_title = '📚 Дашборд нормативных документов РАНХиГС')

### Определяем, в какой кодировке сохранён CSV-файл, с помощью chardet, и уже с ней читаем данные в таблицу. Так меньше шансов напороться на кракозябры.

In [10]:
with open('ranepa_semantic_chunks.csv', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']
    
df = pd.read_csv("ranepa_semantic_chunks.csv", sep = ';', encoding=encoding)

### Добавляем заголовок дашборда через Streamlit. Тут название проекта мы оформили со смайликами - чтобы было красиво.


In [12]:
st.title("📚 Дашборд нормативных документов РАНХиГС")

2025-06-01 18:51:01.227 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

### Создаём сайдбар с заголовком и фильтром по типу документа. Это для того, чтобы можно было выбирать, что показать, а данные автоматически подстраивались под выбор.

In [14]:
st.sidebar.header("🔍 Фильтрация")
doc_types = df['title'].dropna().unique()
selected_type = st.sidebar.multiselect("Тип документа", doc_types, default=doc_types)

filtered_df = df[df['doc_type'].isin(selected_type)]

### Показываем: сколько всего документов, чанков и разных типов документов. Для красоты размещаем их в три колонки. 

In [16]:
col1, col2, col3 = st.columns(3)
col1.metric("📄 Документов", df['doc_id'].nunique())
col2.metric("🔹 Чанков", len(df))
col3.metric("📑 Уникальных типов", df['title'].nunique())

st.markdown("---")

DeltaGenerator()

### Выводим таблицу с первыми 20 чанками. Показываем основные столбцы и таблицу.

In [18]:
st.subheader(" Просмотр чанков")
st.dataframe(filtered_df[['doc_id', 'chunk_id', 'title', 'text']].head(20), use_container_width=True)

DeltaGenerator()

### Строим график - показываем, сколько чанков приходится на каждый тип документа. Данные сначала сортируем по убыванию.

In [20]:
st.subheader("📊 Распределение чанков по документам")
chart_data = df['title'].value_counts().sort_values(ascending=False)
st.bar_chart(chart_data)

DeltaGenerator()

### Считаем длину каждого чанка и строим график, чтобы увидеть, как они распределяются по размеру.

In [22]:
st.subheader("✂️ Распределение длины чанков")
df['chunk_length'] = df['text'].astype(str).apply(len)
st.line_chart(df['chunk_length'])

DeltaGenerator()

### Добавляем поиск по тексту чанков, учитывая регистр и пропуски, чтобы поиск работал так, как нужно.

In [24]:
#Поиск по тексту
st.subheader("🔎 Поиск по ключевому слову")
query = st.text_input("Введите слово или фразу")
if query:
    results = df[df['text'].str.contains(query, case=False, na=False)]
    st.write(f"Найдено {len(results)} фрагментов:")
    st.dataframe(results[['doc_id', 'chunk_id', 'title', 'text']], use_container_width=True)

2025-06-01 18:51:05.509 Session state does not function when running a script without `streamlit run`
